In [1]:
MLFLOW_TRACKING_URI = '../models/mlruns'
MLFLOW_EXPERIMENT_NAME = "skills_jobs_stackoverflow"
MLFLOW_RUN_ID = "a107782f5e814648af1d609b046aa929"

CLUSTERS_YAML_PATH = "../data/processed/features_skills_clusters_description.yaml"

In [2]:
import sys
sys.path.append('../')

In [3]:
import pandas as pd
import sys

sys.path.append('../')
from scripts.JobPrediction import *

# Simulate additional skills

### 1. Calculate base probs

In [95]:
# current_skills = ['C', 'C++']
# current_skills = ['R', 'SQL', 'Docker']
current_skills = ['Python', 'Pandas']

In [96]:
# Initialize model
model = JobPrediction(MLFLOW_TRACKING_URI, MLFLOW_RUN_ID, CLUSTERS_YAML_PATH)

In [97]:
# Calculate base probability
base_predictions = model.predict_jobs_probabilities(current_skills)
base_predictions.sort_values(ascending=False)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished


Academic researcher                              0.360000
Scientist                                        0.333333
Data or business analyst                         0.160000
Engineer, data                                   0.160000
Developer, desktop or enterprise applications    0.100000
Developer, QA or test                            0.096500
Data scientist or machine learning specialist    0.090000
Developer, back-end                              0.090000
Developer, game or graphics                      0.066667
System administrator                             0.053500
Developer, embedded applications or devices      0.040000
Developer, front-end                             0.040000
Developer, full-stack                            0.040000
Database administrator                           0.020000
DevOps specialist                                0.010000
Developer, mobile                                0.000000
dtype: float64

### 2. Calculate Probs for new skills

In [98]:
# Get possible new skills
all_skills = pd.Series(model.get_all_skills())
new_skills = all_skills[~all_skills.isin(current_skills)].copy()

In [99]:
simulated_results = []
for skill in new_skills:
    additional_skill_prob = model.predict_jobs_probabilities([skill] + current_skills)
    additional_skill_uplift = (additional_skill_prob - base_predictions) / base_predictions
    additional_skill_uplift.name = skill
    simulated_results.append(additional_skill_uplift)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_job

In [100]:
simulated_results = pd.DataFrame(simulated_results)

### Recommend

In [101]:
target_job = 'Data scientist or machine learning specialist'

In [102]:
simulated_results[target_job].sort_values(ascending=False)

Keras              3.111111
TensorFlow         2.666667
Torch/PyTorch      2.333333
Hadoop             1.111111
Bash/Shell         1.111111
                     ...   
Groovy            -0.333333
MySQL             -0.333333
Matlab            -0.333333
Microsoft Azure   -0.444444
PyCharm           -0.555556
Name: Data scientist or machine learning specialist, Length: 146, dtype: float64